In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [2]:
import dizx
from dizx import Edge
from dizx import clifford_simplifier as simp
Phase = dizx.CliffordPhase
d = 3
Z = dizx.VertexType.Z
X = dizx.VertexType.X

In [ ]:
c = dizx.Circuit(qudit_amount=2,dim=3)
c.add_gates("H H S",0)
# c.add_gate("CZ",0,1)
# c.add_gates("S S H",1)
# c.add_gates("H S", 0)

In [ ]:
print(c)

In [ ]:
cs = simp.CliffordSimplifier(c)

In [ ]:
cs.dag.children[0].children[0].children

In [ ]:
cs.combine_gates()

In [ ]:
cs.circuit

In [ ]:
cs.dag.children[0].children

In [ ]:
cs.remove_identity_gate()

In [ ]:
cs.push_pauli()

In [ ]:
cs.dag.ch

In [ ]:
cs.push_double_hadamard()

In [ ]:
cs.circuit

In [ ]:
cs.dag.children[0].children

In [33]:
lhs = dizx.Circuit(qudit_amount=1, dim=3)
lhs.add_gates("H S H", 0)
rhs = dizx.Circuit(qudit_amount=1, dim=3)
rhs.add_gates("S S H S S X X", 0)


In [34]:
c = lhs + rhs.adjoint()
c

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), X^-1*(0), X^-1*(0), S^-1*(0), S^-1*(0), HAD^-1*(0), S^-1*(0), S^-1*(0)] gates)

In [81]:
cs = simp.CliffordSimplifier(c)

In [21]:
cs.dag.children[0].children[0].children[0].children[0].children[0].parents

[DAG(node=X(0), children=[DAG(node=X(0), children=[])])]

In [68]:
cs.combine_gates()

True

In [69]:
cs.circuit

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), X^-2*(0), S^-2*(0), HAD^-1*(0), S^-2*(0)] gates)

In [82]:
cs.simple_optimize()
cs.circuit

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), S^-2*(0), X^-4(0)] gates)

In [59]:
i = 0
dag = cs.dag.children[0]
while i<10:
    if len(dag.children) == 0: break
    print("Num children: ", len(dag.children))
    print("First child: ", dag.children[0].node)
    dag = dag.children[0]
    i += 1

Num children:  1
First child:  S(0)
Num children:  1
First child:  HAD(0)
Num children:  1
First child:  X^-2*(0)
Num children:  1
First child:  S^-2*(0)
Num children:  1
First child:  HAD^-1*(0)
Num children:  1
First child:  S^-2*(0)


In [70]:
cs.push_pauli()
#cs.circuit

True

In [71]:
cs.circuit

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), Z^4(0), X^-2*(0), HAD^-1*(0), S^-2*(0)] gates)

In [72]:
cs.push_pauli()

True

In [73]:
cs.circuit

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), Z^4(0), HAD^3*(0), Z^-2(0), S^-2*(0)] gates)

In [74]:
cs.push_pauli()

True

In [75]:
cs.circuit

Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), X^-4(0), Z^-2(0), S^-2*(0)] gates)

In [76]:
print(cs.push_pauli())
cs.circuit

True


Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), Z^-2(0), X^-4(0), S^-2*(0)] gates)

In [77]:
print(cs.push_pauli())
cs.circuit

True


Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), Z^-2(0), S^-2*(0), Z^8(0), X^-4(0)] gates)

In [78]:
print(cs.push_pauli())
cs.circuit

True


Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), S^-2*(0), Z^-2(0), Z^8(0), X^-4(0)] gates)

In [79]:
print(cs.combine_gates())
cs.circuit

True


Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), S^-2*(0), Z^6(0), X^-4(0)] gates)

In [80]:
print(cs.remove_identity_gate())
cs.circuit

True


Circuit[3](1 qudits, 0 dits, [HAD(0), S(0), HAD(0), S^-2*(0), HAD^3*(0), S^-2*(0), X^-4(0)] gates)